# Introduction

In this notebook we use tools and techniques from fastai that we're already very familiar with to construct an image classifier. In the next notebook we'll tell a parallel story based on TensorFlow and its Keras API. 

# Setup

In [ ]:
%matplotlib inline

In [ ]:
from fastai.vision.all import *

Set data dir:

In [ ]:
with open('path.pkl', 'rb') as f:
    path = pickle.load(f)

In [ ]:
path

In [ ]:
path.ls()

# Datablock and dataloaders

![](assets/datablock_api.png)

In [ ]:
flowers = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   splitter=RandomSplitter(0.2),
                   get_y=parent_label,
                   item_tfms=RandomResizedCrop(600),
                   batch_tfms=[*aug_transforms(size=224), 
                               Normalize.from_stats(*imagenet_stats)])

In [ ]:
#flowers.summary(path)

In [ ]:
dls = flowers.dataloaders(path)

In [ ]:
dls.show_batch()

In [ ]:
dls.vocab

# Train a model

In [ ]:
learn = cnn_learner(dls, resnet18, pretrained=True, metrics=[accuracy, error_rate])

## Train only the head of the model

In [ ]:
learn.freeze()

In [ ]:
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(lr.valley))

## Unfreeze and fine-tune the model

In [ ]:
learn.unfreeze()

In [ ]:
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=slice(lr.valley))

# Evaluate the model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
def plot_top_losses_fix(interp, k, largest=True, **kwargs):
        losses,idx = interp.top_losses(k, largest)
        if not isinstance(interp.inputs, tuple): interp.inputs = (interp.inputs,)
        if isinstance(interp.inputs[0], Tensor): inps = tuple(o[idx] for o in interp.inputs)
        else: inps = interp.dl.create_batch(interp.dl.before_batch([tuple(o[i] for o in interp.inputs) for i in idx]))
        b = inps + tuple(o[idx] for o in (interp.targs if is_listy(interp.targs) else (interp.targs,)))
        x,y,its = interp.dl._pre_show_batch(b, max_n=k)
        b_out = inps + tuple(o[idx] for o in (interp.decoded if is_listy(interp.decoded) else (interp.decoded,)))
        x1,y1,outs = interp.dl._pre_show_batch(b_out, max_n=k)
        if its is not None:
            #plot_top_losses(x, y, its, outs.itemgot(slice(len(inps), None)), L(self.preds).itemgot(idx), losses,  **kwargs)
            plot_top_losses(x, y, its, outs.itemgot(slice(len(inps), None)), interp.preds[idx], losses,  **kwargs)
        #TODO: figure out if this is needed
        #its None means that a batch knows how to show itself as a whole, so we pass x, x1
        #else: show_results(x, x1, its, ctxs=ctxs, max_n=max_n, **kwargs)

In [ ]:
plot_top_losses_fix(interp, k=8)